In [2]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent

# CGV

## Event List

In [3]:
def get_CGV_GoodsIdx(event_idx):
    url = "https://m.cgv.co.kr/Event/GiveawayEventDetail.aspx/GetGiveawayEventDetail"
    body = {
        "eventIndex": event_idx,
        "giveawayIndex": None
    }
    response = requests.post(url, json=body)
    text = response.text
    loads = json.loads(text)
    goods_idx = loads["d"]["GiveawayItemList"][0]["GiveawayItemCode"]
    return goods_idx

def get_CGV_EventList(movie_filter='', goods_filter=''):
    url = "https://m.cgv.co.kr/Event/GiveawayEventList.aspx/GetGiveawayEventList"
    body = {
        "theaterCode": "",
        "pageNumber": "1",
        "pageSize": "100"
    }
    response = requests.post(url, json=body)
    text = response.text
    loads = json.loads(text)
    soup = BeautifulSoup(loads['d']['List'])
    event_list = soup.find_all("li")
    event_df = pd.DataFrame()
    p = re.compile(r'\[([\d\D]+)\]') # 영화이름 추출 정규식
    for event in event_list:
        event_idx = event['data-eventidx']
        goods_idx = get_CGV_GoodsIdx(event_idx)
        event_name = event.find("strong").get_text()
        event_period = event.find("span").get_text()
        m = p.match(event_name)
        if m is None:
            movie_name = event_name
            goods_name = event_name
        else:
            movie_name = m.group()[1:-1]
            goods_name = event_name[m.end():] # 띄어쓰기 했다안했다
        event_dict = {"event_idx":event_idx, "goods_idx":goods_idx, "movie":movie_name, "goods":goods_name, "period":event_period}
        event_df = pd.concat([event_df, pd.DataFrame.from_dict(event_dict, orient='index').T], ignore_index=True)
    event_df = event_df[
        (event_df["movie"].str.contains(movie_filter)) &
        (event_df["goods"].str.contains(goods_filter))
    ]
    # 날짜 필터도 추가 예정
    # 위에서 필터링
    return event_df

In [4]:
event_df = get_CGV_EventList()
event_df

,event_idx,goods_idx,movie,goods,period
0,202506236274,2025062362746226,F1 더 무비,4DX 포스터,2025.06.25 ~ 2025.07.22
1,202506236273,2025062362736225,F1 더 무비,IMAX 포스터,2025.06.25 ~ 2025.07.22
2,202506236278,2025062362786229,F1 더 무비,SX 와이드 포스터,2025.06.30 ~ 2025.07.27
3,202506236275,2025062362756227,F1 더 무비,SX 포스터,2025.06.25 ~ 2025.07.22
4,202506186257,2025061862576209,F1 더 무비,TTT,2025.06.25 ~ 2025.07.24
5,202506236276,2025062362766228,F1 더 무비,ULTRA 4DX 모먼트라벨,2025.06.30 ~ 2025.07.27
6,202506266284,2025062662846234,SMG무비갤러리,블리치 지옥편 포스터,2025.06.26 ~ 2025.07.17
7,202507106367,2025071063676310,SMG무비갤러리,하이큐 재능과 센스_북마크,2025.07.11 ~ 2025.07.17
8,202507106368,2025071063686311,SMG무비갤러리,하이큐 재능과 센스_북마크,2025.07.11 ~ 2025.07.17
9,202507046342,2025070463426285,그을린 사랑,북마크,2025.07.09 ~ 2025.07.12


## Goods

- type2: 재고 보유 가능성 높음
- type2.5: 재고 소진 중
- type3: 재고 소진 임박
- type4: 마감

In [7]:
def get_CGV_Theaters(giveawayIndex):
    url = "https://m.cgv.co.kr/Event/GiveawayEventDetail.aspx/GetGiveawayTheaterInfo"
    body = {
        "giveawayIndex": giveawayIndex,
        "areaCode": None
    }
    response = requests.post(url, json=body)
    text = response.text
    loads = json.loads(text)
    AreaList = loads["d"]["AreaList"]
    Theaters = pd.DataFrame()
    for Area in AreaList:
        AreaCode = Area["AreaCode"]
        body = {
            "giveawayIndex": giveawayIndex,
             "areaCode": AreaCode
        }
        response = requests.post(url, json=body)
        text = response.text
        loads = json.loads(text)
        Theater = pd.DataFrame(loads["d"]["TheaterList"])
        Theaters = pd.concat([Theaters, Theater[["TheaterName", "TheaterCode", "CountTypeCode"]]], ignore_index=True)
    return Theaters

In [8]:
giveawayIndex = '2025062362746226'
Theaters = get_CGV_Theaters(giveawayIndex)
Theaters

,TheaterName,TheaterCode,CountTypeCode
0,CGV 강변,0001,type3
1,CGV 고덕강일,0366,type2
2,CGV 방학,0288,type2.5
3,CGV 상봉,0046,type4
4,CGV 신촌아트레온,0150,type2
5,CGV 여의도,0112,type4
6,CGV 연남,0292,type4
7,CGV 영등포,0059,type4
8,CGV 왕십리,0074,type4
9,CGV 용산아이파크몰,0013,type4


In [9]:
def Get_CGV_GoodsStatus(movie_filter="", goods_filter=""):
    event_df = get_CGV_EventList(movie_filter=movie_filter, goods_filter=goods_filter)
    GoodsStatus = pd.DataFrame()
    for idx in event_df.index:
        giveawayIndex = event_df.loc[idx]["goods_idx"]
        Theaters = get_CGV_Theaters(giveawayIndex)
        Theaters["movie"] =  event_df.loc[idx]["movie"]
        Theaters["goods"] =  event_df.loc[idx]["goods"]
        Theaters["period"] =  event_df.loc[idx]["period"]
        GoodsStatus = pd.concat([GoodsStatus, Theaters], ignore_index=True)
    return GoodsStatus

In [10]:
GoodsStatus = Get_CGV_GoodsStatus(movie_filter="F1 더 무비", goods_filter="포스터")
GoodsStatus

,TheaterName,TheaterCode,CountTypeCode,movie,goods,period
0,CGV 강변,0001,type3,F1 더 무비,4DX 포스터,2025.06.25 ~ 2025.07.22
1,CGV 고덕강일,0366,type2,F1 더 무비,4DX 포스터,2025.06.25 ~ 2025.07.22
2,CGV 방학,0288,type2.5,F1 더 무비,4DX 포스터,2025.06.25 ~ 2025.07.22
3,CGV 상봉,0046,type4,F1 더 무비,4DX 포스터,2025.06.25 ~ 2025.07.22
4,CGV 신촌아트레온,0150,type2,F1 더 무비,4DX 포스터,2025.06.25 ~ 2025.07.22
...,...,...,...,...,...,...
91,CGV 천안펜타포트,0110,type2,F1 더 무비,SX 포스터,2025.06.25 ~ 2025.07.22
92,CGV 서면,0005,type2,F1 더 무비,SX 포스터,2025.06.25 ~ 2025.07.22
93,CGV 아시아드,0160,type4,F1 더 무비,SX 포스터,2025.06.25 ~ 2025.07.22
94,CGV 울산삼산,0128,type2,F1 더 무비,SX 포스터,2025.06.25 ~ 2025.07.22


# LotteCinema

In [11]:
def get_LC_EventList():
    url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
    # 특별기획전 "EventClassificationCode":"10"
    body = {
        "ParamList": '{"MethodName":"GetEventLists","channelType":"","osType":"","osVersion":"","EventClassificationCode":"20","SearchText":"","CinemaID":"","PageNo":1,"PageSize":100,"MemberNo":"0"}'
    }
    response = requests.post(url, data=body)
    text = response.text
    loads = json.loads(text)
    event_df = pd.DataFrame(loads["Items"])

    goods_info = [get_LC_GoodsInfo(x) for x in event_df['EventID']]
    event_df = pd.concat([event_df[["EventID", "EventName", "ProgressStartDate", "ProgressEndDate"]], 
                           pd.json_normalize(goods_info)[["FrGiftID", "FrGiftNm"]]],
                         axis=1)

    event_df = event_df[~event_df['FrGiftID'].isna()].reset_index(drop=True)

    p = re.compile(r'\<([\d\D]+)\>') # 영화이름 추출 정규식
    event_df["movie_name"] = [
        '' if p.match(x) is None else p.match(x).group()[1:-1] 
        for x in event_df['EventName']
    ]

    p = re.compile(r'\,([\d\D]+)\)') # 굿즈이름 추출 정규식
    event_df["goods_name"] = [
        '' if len(p.findall(x))==0 else p.findall(x)[0].replace(' ', '')
        for x in event_df['FrGiftNm']
    ]
    return event_df

def get_LC_GoodsInfo(event_idx):
    # EventID, FrGiftID, FrGiftNm
    url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
    body = {
        "ParamList": '{"MethodName":"GetInfomationDeliveryEventDetail","channelType":"","osType":"","osVersion":"","EventID":%s}' % (event_idx)
    }
    response = requests.post(url, data=body)
    text = response.text
    loads = json.loads(text)
    goods_info = loads["InfomationDeliveryEventDetail"][0]["GoodsGiftItems"]
    if len(goods_info) == 0: 
        goods_info = None
    else:
        goods_info = goods_info[0]
    return goods_info

In [25]:
import requests
import json
import pandas as pd
import re

def get_LC_GoodsInfo(event_idx):
    url = "https://www.lottecinema.co.kr/LCWS/Event/EventData.aspx"
    param_json = {
        "MethodName": "GetInfomationDeliveryEventDetail",
        "channelType": "HO",
        "osType": "W",
        "osVersion": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "EventID": str(event_idx)
    }
    files = {
        "paramList": (None, json.dumps(param_json), "application/json")
    }

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Referer": "https://www.lottecinema.co.kr/NLCHS/Event",
        "Origin": "https://www.lottecinema.co.kr",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Connection": "keep-alive",
    }

    with requests.Session() as session:
        # 세션 쿠키 미리 받기
        session.get("https://www.lottecinema.co.kr/NLCHS/Event", headers=headers)
        response = session.post(url, files=files, headers=headers)
        response.raise_for_status()
        data = response.json()

    goods_info = data["InfomationDeliveryEventDetail"][0]["GoodsGiftItems"]
    if not goods_info:
        return None
    return goods_info[0]

def get_LC_EventList(special_exhibition=False):
    url = "https://www.lottecinema.co.kr/LCWS/Event/EventData.aspx"
    param_json = {
        "MethodName": "GetEventLists",
        "channelType": "HO",
        "osType": "W",
        "osVersion": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "EventClassificationCode": "20",
        "SearchText": "",
        "CinemaID": "",
        "PageNo": 1,
        "PageSize": 100,
        "MemberNo": "0"
    }
    if special_exhibition:
        param_json["EventClassificationCode"] = "10"
    files = {
        "paramList": (None, json.dumps(param_json), "application/json")
    }

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Referer": "https://www.lottecinema.co.kr/NLCHS/Event",
        "Origin": "https://www.lottecinema.co.kr",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Connection": "keep-alive",
    }

    with requests.Session() as session:
        session.get("https://www.lottecinema.co.kr/NLCHS/Event", headers=headers)
        response = session.post(url, files=files, headers=headers)
        response.raise_for_status()
        data = response.json()

    event_df = pd.DataFrame(data["Items"])
    print(event_df)
    goods_info = [get_LC_GoodsInfo(x) for x in event_df['EventID']]
    goods_df = pd.json_normalize([g if g else {} for g in goods_info])
    # 결측값 방지
    goods_df = goods_df.reindex(columns=["FrGiftID", "FrGiftNm"]).fillna('')

    event_df = pd.concat(
        [event_df[["EventID", "EventName", "ProgressStartDate", "ProgressEndDate"]], goods_df],
        axis=1
    )

    event_df = event_df[event_df['FrGiftID'] != ''].reset_index(drop=True)

    # 영화 이름 추출 (<> 안 내용)
    p_movie = re.compile(r'<([^<>]+)>')
    event_df["movie_name"] = event_df['EventName'].apply(lambda x: p_movie.search(x).group(1) if p_movie.search(x) else '')

    # 굿즈 이름 추출 (, ... ) 형태에서 추출
    p_goods = re.compile(r',([^,]+)\)')
    event_df["goods_name"] = event_df['FrGiftNm'].apply(lambda x: p_goods.search(x).group(1).strip() if p_goods.search(x) else '')

    return event_df


In [35]:
EventList = get_LC_EventList()

            EventID                                  EventName  \
0   201010048025172             롯시네 보석 발굴 프로젝트 NO.22 <셔터 아일랜드>   
1   201010016925364                        <명탐정 코난>1주차 주말증정이벤트   
2   201010016925362                             <킹 오브 킹스> 무비티켓   
3   201010016925363                           <명탐정코난>1주차 주중이벤트   
4   201010048025173     <극장판 도라에몽: 진구의 그림이야기> 시그니처 무비티켓 증정 이벤트   
5   201010048025159             <극장판 도라에몽: 진구의 그림이야기> 프리미어 상영회   
6   201010016925361                         <시네마천국>2주차 주말증정이벤트   
7   201010048025170             <영화 코바야시네 메이드래곤> 특수관 굿즈 증정 이벤트   
8   201010016925358                           <쥬라기월드>2주차 증정이벤트   
9   201010016925354                            <우교시>1주차현장증정이벤트   
10  201010048025163  <영화 코바야시네 메이드래곤 외로움쟁이 용> SPECIAL ART CARD   
11  201110038525025               <전지적 독자 시점> 얼리버드 프로모션 티켓편 2차   
12  201010016925355                          <시네마 천국>2주차 증정이벤트   
13  201010016925356                        <발코니의 여자들>1주차 증정이벤트   
14  201010

In [36]:
test = get_LC_GoodsInfo("201010016925350")
print(test)

{'EventID': '201010016925350', 'FrGiftID': '12526', 'FrGiftNm': '<슈퍼맨>시그니처 아트카드(7/9수~22화,회차당 수령제한O)'}


In [33]:
def get_LC_Theaters(EventID, FrGiftID):
    url = "https://www.lottecinema.co.kr/LCWS/Event/EventData.aspx"
    param_json = {
        "MethodName":"GetCinemaGoods",
        "channelType":"",
        "osType":"",
        "osVersion":"",
        "EventID":str(EventID), 
        "GiftID":str(FrGiftID)
    }
    
    files = {
        "paramList": (None, json.dumps(param_json), "application/json")
    }
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Referer": "https://www.lottecinema.co.kr/NLCHS/Event",
        "Origin": "https://www.lottecinema.co.kr",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Connection": "keep-alive",
    }
    
    with requests.Session() as session:
        # 세션 쿠키 미리 받기
        session.get("https://www.lottecinema.co.kr/NLCHS/Event", headers=headers)
        response = session.post(url, files=files, headers=headers)
        response.raise_for_status()
        data = response.json()
    Theaters = pd.DataFrame(data["CinemaDivisionGoods"])
    return Theaters

In [37]:
EventID = "201010016925350"
FrGiftID = "12526"
Theaters = get_LC_Theaters(EventID, FrGiftID)
Theaters

,DivisionCode,DetailDivisionCode,CinemaID,CinemaNameKR,CinemaNameUS,SortSequence,Cnt,DetailDivisionNameKR,DetailDivisionNameUS
0,1,0001,1013,가산디지털,Gasan Digital,1,49,서울,Seoul
1,1,0001,1004,건대입구,Konkuk Univ,6,130,서울,Seoul
2,1,0001,1009,김포공항,Gimpo Int'l Airport,7,170,서울,Seoul
3,1,0001,1003,노원,Nowon,8,100,서울,Seoul
4,1,0001,1023,도곡,Dogok,9,0,서울,Seoul
5,1,0001,1017,독산,Doksan,10,0,서울,Seoul
6,1,0001,1012,서울대입구,Seoul Nat'l Univ Station,13,50,서울,Seoul
7,1,0001,1015,신도림,Sindorim,17,30,서울,Seoul
8,1,0001,1007,신림,Sillim,18,110,서울,Seoul
9,1,0001,1001,에비뉴엘(명동),Avenuel,19,30,서울,Seoul


In [38]:
def Get_LC_GoodsStatus(movie_filter="", goods_filter=""):
    event_df = get_LC_EventList()
    event_df = event_df[
        (event_df["EventName"].str.contains(movie_filter)) &
        (event_df["FrGiftNm"].str.contains(goods_filter))
    ]
    GoodsStatus = pd.DataFrame()
    for idx in event_df.index:
        EventID = event_df.loc[idx]["EventID"]
        FrGiftID = event_df.loc[idx]["FrGiftID"]
        Theaters = get_LC_Theaters(EventID, FrGiftID)
        Theaters["movie"] =  event_df.loc[idx]["movie_name"]
        Theaters["goods"] =  event_df.loc[idx]["goods_name"]
        Theaters["period"] =  event_df.loc[idx]["ProgressStartDate"] + '~' + event_df.loc[idx]["ProgressEndDate"]
        GoodsStatus = pd.concat([GoodsStatus, Theaters], ignore_index=True)
    return GoodsStatus

In [39]:
Get_LC_GoodsStatus(goods_filter='엽서')

            EventID                                  EventName  \
0   201010048025172             롯시네 보석 발굴 프로젝트 NO.22 <셔터 아일랜드>   
1   201010016925364                        <명탐정 코난>1주차 주말증정이벤트   
2   201010016925362                             <킹 오브 킹스> 무비티켓   
3   201010016925363                           <명탐정코난>1주차 주중이벤트   
4   201010048025173     <극장판 도라에몽: 진구의 그림이야기> 시그니처 무비티켓 증정 이벤트   
5   201010048025159             <극장판 도라에몽: 진구의 그림이야기> 프리미어 상영회   
6   201010016925361                         <시네마천국>2주차 주말증정이벤트   
7   201010048025170             <영화 코바야시네 메이드래곤> 특수관 굿즈 증정 이벤트   
8   201010016925358                           <쥬라기월드>2주차 증정이벤트   
9   201010016925354                            <우교시>1주차현장증정이벤트   
10  201010048025163  <영화 코바야시네 메이드래곤 외로움쟁이 용> SPECIAL ART CARD   
11  201110038525025               <전지적 독자 시점> 얼리버드 프로모션 티켓편 2차   
12  201010016925355                          <시네마 천국>2주차 증정이벤트   
13  201010016925356                        <발코니의 여자들>1주차 증정이벤트   
14  201010

,DivisionCode,DetailDivisionCode,CinemaID,CinemaNameKR,CinemaNameUS,SortSequence,Cnt,DetailDivisionNameKR,DetailDivisionNameUS,movie,goods,period
0,1,0001,1004,건대입구,Konkuk Univ,6,100,서울,Seoul,시네마 천국,로비엽서,2025.07.09~2025.07.15
1,1,0001,1009,김포공항,Gimpo Int'l Airport,7,50,서울,Seoul,시네마 천국,로비엽서,2025.07.09~2025.07.15
2,1,0001,1003,노원,Nowon,8,50,서울,Seoul,시네마 천국,로비엽서,2025.07.09~2025.07.15
3,1,0001,1002,영등포,Yeongdeungpo,20,49,서울,Seoul,시네마 천국,로비엽서,2025.07.09~2025.07.15
4,1,0001,1014,용산,Yongsan,21,50,서울,Seoul,시네마 천국,로비엽서,2025.07.09~2025.07.15
5,1,0001,1016,월드타워,WorldTower,22,150,서울,Seoul,시네마 천국,로비엽서,2025.07.09~2025.07.15
6,1,0001,1021,은평(롯데몰),Eunpyeong,23,50,서울,Seoul,시네마 천국,로비엽서,2025.07.09~2025.07.15
7,1,0001,1008,청량리,Cheongnyangni,25,50,서울,Seoul,시네마 천국,로비엽서,2025.07.09~2025.07.15
8,1,0002,3008,부평역사,Bupyeong Sation,17,50,경기/인천,Gyeonggi/Incheon,시네마 천국,로비엽서,2025.07.09~2025.07.15
9,1,0002,3041,성남중앙(신흥역),Seongnam Jungang,21,50,경기/인천,Gyeonggi/Incheon,시네마 천국,로비엽서,2025.07.09~2025.07.15


#### 번외 수원 기획전 

In [65]:
EventID = "101010014224011"
url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
body = {
    "ParamList": '{"MethodName":"GetInfomationDeliveryEventDetail","channelType":"","osType":"","osVersion":"","EventID":%s}' % (EventID)
}
response = requests.post(url, data=body)
text = response.text
loads = json.loads(text)
suwon_list = loads["InfomationDeliveryEventDetail"][0]["GoodsGiftItems"]

suwon_list2 = []
for suwon in suwon_list:
    EventID = suwon["EventID"]
    FrGiftID = suwon["FrGiftID"]
    url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
    body = {
        "ParamList": '{"MethodName":"GetCinemaGoods","channelType":"","osType":"","osVersion":"","EventID":%s, "GiftID":%s}' % (EventID, FrGiftID)
    }
    response = requests.post(url, data=body)
    text = response.text
    loads = json.loads(text)
    suwon.update(loads["CinemaDivisionGoods"][0])
    suwon_list2.append(suwon)
    #     print(loads["CinemaDivisionGoods"])
pd.DataFrame(suwon_list2)

,EventID,FrGiftID,FrGiftNm,DivisionCode,DetailDivisionCode,CinemaID,CinemaNameKR,CinemaNameUS,SortSequence,Cnt,DetailDivisionNameKR,DetailDivisionNameUS
0,101010014224011,9183,수원 특별 기획전(스파이더맨 어크로스 더 유니버스),1,0002,3024,수원(수원역),Suwon,23,50,경기/인천,Gyeonggi/Incheon
1,101010014224011,9184,수원 특별 기획전(스파이더맨 노웨이 홈 FUN),1,0002,3024,수원(수원역),Suwon,23,100,경기/인천,Gyeonggi/Incheon
2,101010014224011,9185,수원 특별 기획전(그란투리스모),1,0002,3024,수원(수원역),Suwon,23,350,경기/인천,Gyeonggi/Incheon
3,101010014224011,9186,수원 특별 기획전(탑건: 매버릭),1,0002,3024,수원(수원역),Suwon,23,850,경기/인천,Gyeonggi/Incheon
4,101010014224011,9187,수원 특별 기획전(언어의 정원),1,0002,3024,수원(수원역),Suwon,23,350,경기/인천,Gyeonggi/Incheon
5,101010014224011,9188,수원 특별 기획전(날씨의 아이),1,0002,3024,수원(수원역),Suwon,23,100,경기/인천,Gyeonggi/Incheon


# Megabox

In [53]:
def get_MB_EventList():
    url = "https://www.megabox.co.kr/on/oh/ohe/Event/eventMngDiv.do"
    body = {
        "currentPage": "1",
        "recordCountPerPage": "1000",
        "eventStatCd": "ONG",
        "eventTitle": "",
        "eventDivCd": "CED03",
        "eventTyCd": "",
        "orderReqCd": "ONGlist"
    }
    ua = UserAgent()
    headers = {
        'User-Agent': ua.random
    }
    response = requests.post(url, json=body, headers=headers)
    text = response.text
    soup = BeautifulSoup(text)

    eventBtn_list = soup.find_all("a", class_="eventBtn")
    for x in eventBtn_list:
        try:
            event_list.append({
                "event_no": x["data-no"],
                "event_name": x.get("title", ""),
                "period": x.find("p", class_="date").get_text(strip=True)
            })
        except Exception as e:
            print(f"이벤트 파싱 실패: {e}")
    event_df = pd.DataFrame(event_list)
    goods_info = [get_MB_GoodsInfo(x) for x in event_df['event_no']]
    event_df = pd.concat([event_df, pd.json_normalize(goods_info)], axis=1)
    return event_df


def get_MB_GoodsInfo(event_idx):
    url = f"https://www.megabox.co.kr/event/detail?eventNo={event_idx}"
    ua = UserAgent()
    headers = {
        'User-Agent': ua.random
    }
    response = requests.post(url, headers=headers)
    text = response.text
    soup = BeautifulSoup(text)
    goods_tag = soup.find("button", attrs={"id":"btnSelectGoodsStock"})
    if goods_tag is None:
        goods_info = None
    else:
        goods_info = {
            "data-nm": goods_tag["data-nm"],
            "data-pn": goods_tag["data-pn"]
        }
    return goods_info

In [63]:
event_df = get_MB_EventList()
event_df

,event_no,event_name,period,data-nm,data-pn
0,18254,메가박스 아트그라피 <킹 오브 킹스> 상세보기,2025.07.19 ~ 2025.07.20,NaN,NaN
1,18269,[프로모션][굿즈] <명탐정 코난: 척안의 잔상> 스트랩키링 굿즈 상세보기,2025.07.16 ~ 2025.08.16,NaN,NaN
2,18252,메가박스 드로잉 카드 No.50 <스왈로우테일 버터플라이> 상세보기,2025.07.16 ~ 2025.08.12,&amp;lt;&igrave;&curren;&igrave;&euml;&iexc...,FG000380
3,18251,메가박스 드로잉 카드 No.49 <킹 오브 킹스> 상세보기,2025.07.16 ~ 2025.08.12,&amp;lt;킹 오브 킹스&amp;gt; 드로잉 카드,FG000379
4,18267,"[프로모션][굿즈] <킹 오브 킹스 > 파우치, 키링 굿즈 상세보기",2025.07.16 ~ 2025.07.31,NaN,NaN
...,...,...,...,...,...
124,16048,메가박스앱 첫 구매고객 대상 1+1 쿠폰 이벤트! 상세보기,2025.01.30 ~ 2026.01.06,NaN,NaN
125,17006,빵원티켓이 달라졌어요! 상세보기,2025.01.14 ~ 2025.12.31,NaN,NaN
126,16920,나도 영화 평론가! 관람평 작성하고 50P 받자! 상세보기,2025.01.01 ~ 2025.12.31,NaN,NaN
127,17646,[100% 누구나 당첨] 알림받으면 최대 영화 무료 관람! 상세보기,2024.12.02 ~ 2025.12.31,NaN,NaN


In [62]:
event_df

,event_no,event_name,period,data-nm,data-pn
0,18254,메가박스 아트그라피 <킹 오브 킹스> 상세보기,2025.07.19 ~ 2025.07.20,NaN,NaN
1,18269,[프로모션][굿즈] <명탐정 코난: 척안의 잔상> 스트랩키링 굿즈 상세보기,2025.07.16 ~ 2025.08.16,NaN,NaN
2,18252,메가박스 드로잉 카드 No.50 <스왈로우테일 버터플라이> 상세보기,2025.07.16 ~ 2025.08.12,&amp;lt;스왈로우테일 버터플라이&amp;gt;드로잉,FG000380
3,18251,메가박스 드로잉 카드 No.49 <킹 오브 킹스> 상세보기,2025.07.16 ~ 2025.08.12,&amp;lt;킹 오브 킹스&amp;gt; 드로잉 카드,FG000379
4,18267,"[프로모션][굿즈] <킹 오브 킹스 > 파우치, 키링 굿즈 상세보기",2025.07.16 ~ 2025.07.31,NaN,NaN
...,...,...,...,...,...
81,16048,메가박스앱 첫 구매고객 대상 1+1 쿠폰 이벤트! 상세보기,2025.01.30 ~ 2026.01.06,NaN,NaN
82,17006,빵원티켓이 달라졌어요! 상세보기,2025.01.14 ~ 2025.12.31,NaN,NaN
83,16920,나도 영화 평론가! 관람평 작성하고 50P 받자! 상세보기,2025.01.01 ~ 2025.12.31,NaN,NaN
84,17646,[100% 누구나 당첨] 알림받으면 최대 영화 무료 관람! 상세보기,2024.12.02 ~ 2025.12.31,NaN,NaN


In [56]:
event_df[~event_df['data-pn'].isna()].sort_values('data-pn')

,event_no,event_name,period,data-nm,data-pn
70,17992,메가박스 드로잉 카드 No.45 <태양의 노래> 상세보기,2025.06.11 ~ 2025.07.22,&amp;lt;태양의 노래&amp;gt; 드로잉 카드,FG000361
27,17992,메가박스 드로잉 카드 No.45 <태양의 노래> 상세보기,2025.06.11 ~ 2025.07.22,&amp;lt;태양의 노래&amp;gt; 드로잉 카드,FG000361
68,18103,<엘리오> 돌비 포스터 증정 이벤트 상세보기,2025.06.20 ~ 2025.07.13,[엘리오] 돌비포스터,FG000364
25,18103,<엘리오> 돌비 포스터 증정 이벤트 상세보기,2025.06.20 ~ 2025.07.13,[엘리오] 돌비포스터,FG000364
65,18073,메가박스 드로잉 카드 No.47 <그을린 사랑> 상세보기,2025.06.25 ~ 2025.07.29,&amp;lt;그을린 사랑&amp;gt; 드로잉 카드,FG000366
22,18073,메가박스 드로잉 카드 No.47 <그을린 사랑> 상세보기,2025.06.25 ~ 2025.07.29,&amp;lt;&ecirc;&middot;&cedil;&igrave;&euml;...,FG000366
64,18074,메가박스 드로잉 카드 No.48 <노이즈> 상세보기,2025.06.25 ~ 2025.07.29,&amp;lt;&euml; &cedil;&igrave;&acute;&igrave;...,FG000367
21,18074,메가박스 드로잉 카드 No.48 <노이즈> 상세보기,2025.06.25 ~ 2025.07.29,&amp;lt;노이즈&amp;gt; 드로잉 카드,FG000367
31,18133,메가박스 오리지널 티켓 Re.34 <라이언 일병 구하기> 상세보기,2025.06.23 ~ 2025.07.19,&amp;lt;라이언 일병 구하기&amp;gt; 오리지널 티켓,FG000368
74,18133,메가박스 오리지널 티켓 Re.34 <라이언 일병 구하기> 상세보기,2025.06.23 ~ 2025.07.19,&amp;lt;&euml;&frac14;&igrave;&acute;&igrave...,FG000368


In [57]:
def get_MB_Theaters(goodsNo):
    url = "https://www.megabox.co.kr/on/oh/ohe/Event/selectGoodsStockPrco.do"
    body = {
        "goodsNo": goodsNo
    }
    ua = UserAgent()
    headers = {
        'User-Agent': ua.random
    }
    response = requests.post(url, data=body, headers=headers)
    text = response.text
    soup = BeautifulSoup(text)
    brch_list = soup.find_all("li", class_="brch")
    theater_list = [ 
        { 
            "theater": x.find("a").get_text(),
            "status": x.find("span").get_text()        
        }
        for x in brch_list
    ]
    Theaters = pd.DataFrame(theater_list)
    return Theaters

In [58]:
def Get_MB_GoodsStatus(goods_filter=""):
    event_df = get_MB_EventList()
    event_df = event_df[~event_df['data-pn'].isna()].sort_values('data-pn')
    event_df = event_df[
        (event_df["data-nm"].str.contains(goods_filter))
    ]
    GoodsStatus = pd.DataFrame()
    for idx in event_df.index:
        goodsNo = event_df.loc[idx]["data-pn"]
        Theaters = get_MB_Theaters(goodsNo)
        Theaters["event_name"] =  event_df.loc[idx]["event_name"]
        Theaters["data-nm"] =  event_df.loc[idx]["data-nm"]
        Theaters["period"] =  event_df.loc[idx]["period"]
        GoodsStatus = pd.concat([GoodsStatus, Theaters], ignore_index=True)
    return GoodsStatus

In [59]:
Get_MB_GoodsStatus()

,theater,status,event_name,data-nm,period
0,강남,보유,메가박스 드로잉 카드 No.45 <태양의 노래> 상세보기,&amp;lt;태양의 노래&amp;gt; 드로잉 카드,2025.06.11 ~ 2025.07.22
1,강동,보유,메가박스 드로잉 카드 No.45 <태양의 노래> 상세보기,&amp;lt;태양의 노래&amp;gt; 드로잉 카드,2025.06.11 ~ 2025.07.22
2,구의이스트폴,소진,메가박스 드로잉 카드 No.45 <태양의 노래> 상세보기,&amp;lt;태양의 노래&amp;gt; 드로잉 카드,2025.06.11 ~ 2025.07.22
3,군자,소진,메가박스 드로잉 카드 No.45 <태양의 노래> 상세보기,&amp;lt;태양의 노래&amp;gt; 드로잉 카드,2025.06.11 ~ 2025.07.22
4,더부티크목동현대백화점,소진,메가박스 드로잉 카드 No.45 <태양의 노래> 상세보기,&amp;lt;태양의 노래&amp;gt; 드로잉 카드,2025.06.11 ~ 2025.07.22
...,...,...,...,...,...
1463,송도(트리플스트리트),보유,메가박스 드로잉 카드 No.50 <스왈로우테일 버터플라이> 상세보기,&amp;lt;스왈로우테일 버터플라이&amp;gt;드로잉,2025.07.16 ~ 2025.08.12
1464,대전,보유,메가박스 드로잉 카드 No.50 <스왈로우테일 버터플라이> 상세보기,&amp;lt;스왈로우테일 버터플라이&amp;gt;드로잉,2025.07.16 ~ 2025.08.12
1465,대구신세계(동대구),보유,메가박스 드로잉 카드 No.50 <스왈로우테일 버터플라이> 상세보기,&amp;lt;스왈로우테일 버터플라이&amp;gt;드로잉,2025.07.16 ~ 2025.08.12
1466,사상,준비중,메가박스 드로잉 카드 No.50 <스왈로우테일 버터플라이> 상세보기,&amp;lt;스왈로우테일 버터플라이&amp;gt;드로잉,2025.07.16 ~ 2025.08.12


In [60]:
# 이벤트 목록에 없는 굿즈 정보 확인

for i in range(377, 385):
    
    goodsNo = "FG000"+str(i)

    url = "https://www.megabox.co.kr/on/oh/ohe/Event/selectGoodsStockPrco.do"
    body = {
        "goodsNo": goodsNo
    }
    ua = UserAgent()
    headers = {
        'User-Agent': ua.random
    }
    response = requests.post(url, data=body, headers=headers)
    text = response.text
    soup = BeautifulSoup(text)
    # brch_list = soup.find_all("li", class_="brch")
    # theater_list = [ 
    #     { 
    #         "theater": x.find("a").get_text(),
    #         "status": x.find("span").get_text()        
    #     }
    #     for x in brch_list
    # ]

    print(goodsNo, soup.find("div", class_="tit").get_text())
    # theater_list

FG000377 <슈퍼맨> 오리지널 티켓
FG000378 <슈퍼맨> 오리지널 슬라이드
FG000379 <킹 오브 킹스> 드로잉 카드
FG000380 <스왈로우테일 버터플라이>드로잉
FG000381 엔하이픈 VR 굿즈
FG000382 
FG000383 
FG000384 
